In [9]:
#Importing packages
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from dotenv import load_dotenv
import psycopg2

In [10]:
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

In [11]:
try:
    conn = psycopg2.connect(DATABASE_URL)
    print("Connected to the database")
    cursor = conn.cursor()
    sql = """ SELECT id,name FROM \"Tags\" """
    cursor.execute(sql)
    db_tags = cursor.fetchall()
    tags = {}
    for i in db_tags:
        tags[i[0]] = i[1]
     
    #for i in tags:
    feature_list =["uuid"]+[i[0] for i in db_tags]
    df = pd.DataFrame(columns=feature_list)
    sql = """  SELECT \"poemId\",\"tagId\" FROM  \"PoemsTags\" """
    cursor.execute(sql)
    poems_tags = cursor.fetchall()
 

    
    dict_poems_tags = {}
    for i in poems_tags:
        if i[0] in dict_poems_tags:
            dict_poems_tags[i[0]][i[1]] = 1
        else:
            dict_poems_tags[i[0]] = {}
            dict_poems_tags[i[0]]['uuid'] = i[0]
            dict_poems_tags[i[0]][i[1]] = 1
    
    for i in dict_poems_tags:
        df.loc[len(df)] =   dict_poems_tags[i]
        

    df = df.fillna(0)
    #df.to_csv('model.csv',index=False)
except Exception as e:
    print("Database Error",e)
finally:
    conn.close()

Connected to the database


In [31]:
remove_uuid = df.drop(['uuid'],axis=1)
X = remove_uuid

post_data = {1:1,2:1,3:1,4:1,5:1}
fill_data = {}
for i in tags:
    if i in post_data:
        fill_data[i] = 1
    else:
        fill_data[i] = 0
post = pd.DataFrame(data=fill_data,index=[0])
neigh = NearestNeighbors(n_neighbors=10, algorithm='brute')
neigh.fit(X)
distances,indices = neigh.kneighbors(post)
conn = psycopg2.connect(DATABASE_URL)
print("Connected to the database")
cursor = conn.cursor()
for i in range(len(distances.flatten())):
    #  print('{0}: {1}, with a distance of {2}.'.format(i+1, movies['Movie Name'].iloc[indices.flatten()[i]],distances.flatten()[i]))
    uuid = (df['uuid'].iloc[indices.flatten()[i]])
    
    sql = """ SELECT title FROM  \"Poems\" WHERE id=%s """
    values = (uuid,)
    cursor.execute(sql,values)
    info = cursor.fetchone()
    print(info,distances.flatten()[i])
conn.close()
    

Connected to the database
('Falsa elegía',) 0.0
('Rincón de la sangre',) 2.449489742783178
('Quédate callado',) 2.449489742783178
('Mejor querría ser horizontal…',) 2.449489742783178
('Te vi crecer como un árbol…',) 2.449489742783178
('Llanto por el prefecto de Vin Tuong',) 2.449489742783178
('El último deseo',) 2.449489742783178
('Canción',) 2.449489742783178
('Veo que el ciruelo de invierno ha brotado',) 2.449489742783178
('Oí decir a los muy, muy viejos',) 2.449489742783178
